In [ ]:
pip install Orange3

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
# import category encoders
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import precision_score
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_classification
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.evaluate import paired_ttest_5x2cv
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from scipy import stats as stats
from scipy.stats import rankdata
from sklearn.linear_model import LogisticRegression
#from orange3.evaluation import compute_CD, graph_ranks
import Orange as ora
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19final-dataset/both_test_unbalanced.csv
/kaggle/input/covid19final-dataset/pcr_unbalanced.csv
/kaggle/input/covid19final-dataset/rapid_balanced.csv
/kaggle/input/covid19final-dataset/both_test_balanced.csv
/kaggle/input/covid19final-dataset/rapid_unbalanced.csv
/kaggle/input/covid19final-dataset/pcr_balanced.csv


# Comparing multiple classifiers over multiple datasets

In [ ]:
#balanced data
data_rapid = pd.read_csv("/kaggle/input/covid19final-dataset/rapid_balanced.csv")
data_rapid=data_rapid.astype(int)

data_pcr= pd.read_csv("/kaggle/input/covid19final-dataset/pcr_balanced.csv")
data_pcr=data_pcr.astype(int)

data_both= pd.read_csv("/kaggle/input/covid19final-dataset/both_test_balanced.csv")
data_both=data_both.astype(int)


#unbalanced data
data_rapid_unb = pd.read_csv("/kaggle/input/covid19final-dataset/rapid_unbalanced.csv")
data_rapid_unb=data_rapid_unb.astype(int)


data_pcr_unb= pd.read_csv("/kaggle/input/covid19final-dataset/pcr_unbalanced.csv")
data_pcr_unb=data_pcr_unb.astype(int)


both_data_unb = pd.read_csv("/kaggle/input/covid19final-dataset/both_test_unbalanced.csv")


In [ ]:
def RandomForest_classif(x_train,y_train,param):
    #Classification
    
    
    clf= RandomForestClassifier(**param)
    clf=clf.fit(x_train, y_train)
    
    return clf

def Kneighbors_classif(x_train,y_train,param):
    #Classification
    

    clf= KNeighborsClassifier(**param)
    clf= clf.fit(x_train, y_train)

    return clf

def DecisionTree_classif(x_train,y_train,param):
    #Classification
    
    clf = tree.DecisionTreeClassifier(**param)
    clf = clf.fit(x_train,y_train)
    
    return clf

def mpl_classif(x_train,y_train,param):
    
    clf =  MLPClassifier(**param)
    clf=clf.fit(x_train,y_train)
    
    return clf

def gb_classif(x_train,y_train,param):
    
    clf=GradientBoostingClassifier(**param)
    clf=clf.fit(x_train, y_train)
    
    return clf

def xgb_classif(x_train,y_train,param):
    clf = xgb.XGBClassifier(**param)
    
    return clf.fit(x_train, y_train)

def svc_classif(x_train,y_train,param):
    
    regr = svm.SVC(**param)
    regr=regr.fit(x_train, y_train)
    
    return regr

def lg_with_regu_classif(x_train,y_train,param):
    
    clf = LogisticRegression(** param)
    clf=clf.fit(x_train,y_train)
    
    return clf
def lg_without_regu_classif(x_train,y_train,param):
    
    clf = LogisticRegression(** param)
    clf=clf.fit(x_train,y_train)
   
    return clf

In [ ]:
def calculate_metrics(x_train, x_test,y_train, y_test,clf):
    
        #prediction
        y_pred=clf.predict(x_test)
        
        
        #confusion matrix
        tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()
        
        
        #recall
        recall=(tp/(tp+fn))*100
        
        
        return recall

In [ ]:
def k_fold_cross_validation(x,y,lista_param):
    dt=pd.DataFrame(columns=['DT','RF','GBM','XGBoost','Mlp','SVM','KNN','LRR','LRR'],index=range(50))
    
    kf=RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
                        
    
    kf.get_n_splits(x,y)
    k=0
    for train_index, test_index in kf.split(x,y):

        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        clf_dt=DecisionTree_classif(x_train,y_train,lista_param[0])
        dt.iloc[k:k+1,:1]=calculate_metrics(x_train, x_test,y_train, y_test,clf_dt)
        
        clf_rf=RandomForest_classif(x_train,y_train,lista_param[1])
        dt.iloc[k:k+1,1:2]=calculate_metrics(x_train, x_test,y_train, y_test,clf_rf)
        
        clf_svm=svc_classif(x_train,y_train,lista_param[2])
        dt.iloc[k:k+1,5:6]=calculate_metrics(x_train, x_test,y_train, y_test,clf_svm)
       
        clf_knn=Kneighbors_classif(x_train,y_train,lista_param[3])
        dt.iloc[k:k+1,6:7]=calculate_metrics(x_train, x_test,y_train, y_test,clf_knn)
        
        clf_gbm=gb_classif(x_train,y_train,lista_param[4])
        dt.iloc[k:k+1,2:3]=calculate_metrics(x_train, x_test,y_train, y_test,clf_gbm)
        
        clf_lg_regu=lg_with_regu_classif(x_train,y_train,lista_param[5])
        dt.iloc[k:k+1,7:8]=calculate_metrics(x_train, x_test,y_train, y_test,clf_lg_regu)
        
        clf_lg=lg_without_regu_classif(x_train,y_train,lista_param[6])
        dt.iloc[k:k+1,8:9]=calculate_metrics(x_train, x_test,y_train, y_test,clf_lg)
        
        clf_mlp=mpl_classif(x_train,y_train,lista_param[7])
        dt.iloc[k:k+1,4:5]=calculate_metrics(x_train, x_test,y_train, y_test,clf_mlp)
        
        clf_xgboost=xgb_classif(x_train,y_train,lista_param[8])
        dt.iloc[k:k+1,3:4]=calculate_metrics(x_train, x_test,y_train, y_test,clf_xgboost)
        print(k)
        k+=1
        
        
        
    return  dt

In [ ]:
def mean_performace(dt):
    # Then, we extract the performances as a numpy.ndarray.
    performances_array =  dt.iloc[:,:].values
    algorithms_names=dt.columns.values
    # Finally, we apply the Friedman test.
    t, p =stats.friedmanchisquare(*performances_array)
    # summarize
    print('P-value:' , p, 't-Statistic: %.3f' % (t))
    # interpret the result
    if p <= 0.1:
        print('Difference between mean performance is probably real')
    else:
        print('Algorithms probably have the same performance')

In [ ]:
def calculate_average_ranks(dt):
    performances_array =  dt.iloc[:,:].values
    algorithms_names=dt.columns.values
    # Calculating the ranks of the algorithms for each dataset. The value of p is multipled by -1
    # because the rankdata method ranks from the smallest to the greatest performance values.
    # Since we are considering Recall as our performance measure, we want larger values to be best ranked.
    ranks = np.array([rankdata(-p) for p in performances_array])
    # Calculating the average ranks.
    average_ranks = np.mean(ranks, axis=0)
    print('\n'.join('{} average rank: {}'.format(a, r) for a, r in zip(algorithms_names, average_ranks)))
    return average_ranks

In [ ]:
def plot_critical_difference(dt,average_ranks,name):
    # This method computes the critical difference for Nemenyi test with alpha=0.1.
    # For some reason, this method only accepts alpha='0.05' or alpha='0.1'.
    algorithms_names=dt.columns.values
    cd =  ora.evaluation.compute_CD(average_ranks,
    n=len(dt),
    alpha='0.1',
    test='nemenyi')
    # This method generates the plot.
    ora.evaluation.graph_ranks(average_ranks,
    names=algorithms_names,
    cd=cd,
    width=10,
    textspace=1.5,
    reverse=True)
    plt.savefig(name)
    plt.show()


# rapid balanced test

In [ ]:

lista_param=[
             {},
             {},
             {'kernel': 'rbf', 'C': 6,'probability':True},
             {'weights': 'distance', 'n_neighbors': 15,'n_jobs': None, 'leaf_size': 5, 'algorithm': 'ball_tree'},
             {'n_estimators':500,'max_depth':5},
             {'solver': 'liblinear', 'penalty': 'l2','C': 0.1},
             {'solver': 'liblinear', 'penalty': 'l2','C': 1.0},
             {'solver': 'adam', 'random_state': 1, 'max_iter': 1200, 'learning_rate': 'adaptive','alpha': 0.0001, 'activation': 'relu'},
             {'n_estimators':300,'max_depth':9,'min_child_weight': 2}]

d_rapid=k_fold_cross_validation(data_rapid.iloc[:,0:10],data_rapid['Class'],lista_param) 

mean_performace(d_rapid)
d_rapid_average_ranks=calculate_average_ranks(d_rapid)
plot_critical_difference(d_rapid,d_rapid_average_ranks,'model_evaluation_rapid_balanced.jpg')


# This method computes the critical difference for Bonferroni-Dunn test with alpha=0.1.
# For some reason, this method only accepts alpha='0.05' or alpha='0.1'.
cd = ora.evaluation.compute_CD(average_ranks,
n=len(dt),
alpha='0.1',
test='bonferroni-dunn')
# This method generates the plot.
ora.evaluation.graph_ranks(average_ranks,
names=algorithms_names,
cd=cd,
cdmethod=0,
width=10,
textspace=1.5,
reverse=True)
plt.show()


# rapid unbalanced test

In [ ]:
'''
lista_param=[
{},
{},
{'kernel': 'rbf', 'C': 11,'probability':True},
{'weights': 'distance', 'n_neighbors': 5, 'n_jobs': None, 'leaf_size': 3, 'algorithm': 'kd_tree'},
{'n_estimators':500,'max_depth':5},
{'solver': 'saga', 'penalty': 'l2', 'max_iter': 1200, 'C': 0.1},
{'solver': 'lbfgs', 'penalty': 'l2', 'max_iter': 400, 'C': 4.0},
{'max_iter': 1800,'solver':'lbfgs', 'alpha':1e-5, 'random_state':42},
{'n_estimators':300,'max_depth':9,'min_child_weight': 2}]          

d_rapid_unb=k_fold_cross_validation(data_rapid_unb.iloc[:,0:10],data_rapid_unb['Class'],lista_param) 

mean_performace(d_rapid_unb)
d_rapid_unb_average_ranks=calculate_average_ranks(d_rapid_unb)
plot_critical_difference(d_rapid_unb,d_rapid_unb_average_ranks,'model_evaluation_rapid_unbalanced.jpg')
'''

# pcr balanced test

In [ ]:
'''
lista_param=[
{},
{},
{'kernel': 'rbf', 'C': 7,'probability':True},
{'weights': 'distance', 'n_neighbors': 5, 'n_jobs': None, 'leaf_size': 3, 'algorithm': 'kd_tree'},
{'n_estimators':500,'max_depth':5},
{'solver': 'liblinear', 'penalty': 'l2','C': 0.1},
{'solver': 'liblinear', 'penalty': 'l2','C': 1.0},
{'max_iter': 1800,'solver':'lbfgs', 'alpha':1e-5, 'random_state':42},
{'n_estimators':300,'max_depth':9,'min_child_weight': 2}]

pcr_bal=k_fold_cross_validation(data_pcr.iloc[:,0:10],data_pcr['Class'],lista_param) 

mean_performace(pcr_bal)
d_pcr_bal_average_ranks=calculate_average_ranks(pcr_bal)
plot_critical_difference(pcr_bal,d_pcr_bal_average_ranks,'model_evaluation_pcr_balanced.jpg')
'''

# pcr unbalanced test

In [ ]:
'''
lista_param=[
{},
{},
{'kernel': 'rbf', 'C': 6,'probability':True},
{'weights': 'distance', 'n_neighbors': 8, 'n_jobs': -1, 'leaf_size': 2, 'algorithm': 'auto'},
{'n_estimators':500,'max_depth':5},
{'solver': 'liblinear', 'penalty': 'l2','C': 0.1},
{'solver': 'liblinear', 'penalty': 'l2','C': 1.0},
{'max_iter': 1800,'solver':'lbfgs', 'alpha':1e-5, 'random_state':42},
{'n_estimators':300,'max_depth':9,'min_child_weight': 2}]

pcr_unb=k_fold_cross_validation(data_pcr_unb.iloc[:,0:10],data_pcr_unb['Class'],lista_param) 

mean_performace(pcr_unb)
d_pcr_unb_average_ranks=calculate_average_ranks(pcr_unb)
plot_critical_difference(pcr_unb,d_pcr_unb_average_ranks,'model_evaluation_pcr_unbalanced.jpg')
'''

# both balanced test

In [ ]:
'''
lista_param=[{},{},{'kernel': 'rbf', 'C': 9,'probability':True},
             {'weights': 'distance','n_neighbors': 8, 'n_jobs': -1,'leaf_size': 5, 'algorithm': 'brute'},
             {'n_estimators':500,'max_depth':5},
             {'solver': 'liblinear', 'penalty': 'l2','C': 0.1},
             {'solver': 'liblinear', 'penalty': 'l2','C': 1.0},
             {'max_iter': 1200,'solver':'lbfgs', 'alpha':1e-5, 'random_state':42},
            {'n_estimators':300,'max_depth':9,'min_child_weight': 2}]


both_test=k_fold_cross_validation(data_both.iloc[:,0:10],data_both['Class'],lista_param) 

mean_performace(both_test)
both_test_average_ranks=calculate_average_ranks(both_test)
plot_critical_difference(both_test,both_test_average_ranks,'model_evaluation_bothtest_balanced.jpg')
'''

# both unbalanced test

In [ ]:
'''
lista_param=[{},{},{'kernel': 'rbf', 'C': 8,'probability':True},
             {'n_neighbors': 15},
             {'n_estimators':500,'max_depth':5},
             {'solver': 'liblinear', 'penalty': 'l2','C': 0.1},
             {'solver': 'liblinear', 'penalty': 'l2','C': 1.0},
             {'max_iter': 1800,'solver':'lbfgs', 'alpha':1e-5, 'random_state':42},
            {'n_estimators':300,'max_depth':9,'min_child_weight': 2}]


both_test_unb=k_fold_cross_validation(both_data_unb.iloc[:,0:10],both_data_unb['Class'],lista_param) 

mean_performace(both_test_unb)
both_test_unb_average_ranks=calculate_average_ranks(both_test_unb)
plot_critical_difference(both_test_unb,both_test_unb_average_ranks,'model_evaluation_bothtest_unbalanced.jpg')
'''